# Visualization of electrostatics

In this lab, we will visualize the electrostatic potential from APBS.

#Part 0 – Downloading and Installing the required software

We will first install **py3Dmol** and **openbabel**. The latter will allow us to convert between file formats.

In [ ]:
try:
  import py3Dmol
except:
  !pip install py3Dmol
  import py3Dmol

In [ ]:
#Install conda using the conda-colab library
!pip install -q condacolab
import condacolab
condacolab.install_miniconda()
!conda update -n base -c defaults conda

#Install openbabel
!conda install -c conda-forge openbabel --yes

Next, we will download and decompress the atomic coordinates and electrostatic potential into the hosted runtime.

In [ ]:
!wget -L https://raw.githubusercontent.com/CCBatIIT/modelingworkshop/main/labs-complete/2-1/k5f10lg2t7_20220306/k5f10lg2t7.pqr
!wget -L https://raw.githubusercontent.com/CCBatIIT/modelingworkshop/main/labs-complete/2-1/k5f10lg2t7_20220306/k5f10lg2t7-pot.dx.gz

import os
if not os.path.isfile('k5f10lg2t7-pot.dx'):
  !gunzip k5f10lg2t7-pot.dx.gz

pqr = open('k5f10lg2t7.pqr', 'r').read()

# Part I - Visualizing protonation

Let's look at all the histidines in the pqr file. What are the residue names?

In [ ]:
for line in pqr.split('\n'):
  if line.find('HI')>-1 and line.find('CA')>-1:
    print(line)

Now look at the structure of a few histidines. The view below shows the protons on the histidine but not other protein aotms. Change the `resi` and `chain` variables below to zoom in to a different residue. Can you tell why a certain imidazole is stabilized?

In [ ]:
resi = '163'
chain = 'A'

F = open('resi.pqr','w')
for line in pqr.split('\n'):
  if line[21:26] == f'{chain}{resi:>4s}':
    F.write(line + '\n')
F.close()

!obabel -ipqr resi.pqr -omol -O resi.mol 2> conversion.log
mol = open('resi.mol', 'r').read()

view = py3Dmol.view()
view.setBackgroundColor('white')
view.addModel(mol, 'mol')
view.setStyle({}, {'stick': {'colorscheme':'element'}})
view.addModel(pqr,'pdb')
sel = {'within':{'distance':'7', 'sel':{'and':[{'resi':resi}, {'chain':chain}]}}}
view.setStyle(sel, {'stick': {'colorscheme':'element'}})
view.zoomTo(sel)
view.show()

# Part II - Visualizing electrostatics

I was unable to get py3Dmol to show volumetric data in Google Colab. As an alternative, let's visualize the results with plot.ly. 

First, we will read the OpenDX file from APBS.

In [ ]:
import numpy as np
F = open('k5f10lg2t7-pot.dx','r')

# Read the header
line = F.readline()
while line.find('object') == -1:
  line = F.readline()
header = {}
header['counts'] = [int(x) for x in line.split(' ')[-3:]]
for name in ['origin', 'd0', 'd1', 'd2']:
  header[name] = [float(x) for x in F.readline().split(' ')[-3:]]
F.readline()
header['npts'] = int(F.readline().split(' ')[-3])

# Read the data
vals = np.ndarray(shape=header['npts'], dtype=float)
index = 0
while index < header['npts']:
  line = F.readline()[:-1]
  items = [float(item) for item in line.split()]
  vals[index:index + len(items)] = items
  index = index + len(items)

F.close()


The file is too unwieldy to run on Google Colab. We will only use a subset of points

In [ ]:
print(vals.shape)
vals = vals.reshape(header['counts'])
vals = vals[::5,::5,::5]
print(vals.shape)

Now we will visualize a series of isosurfaes using plotly. Where is the electrostatic potential positive and negative?

In [ ]:
def enable_plotly_in_cell(): # define once
  import IPython
  from plotly.offline import init_notebook_mode
  display(IPython.core.display.HTML('''<script src="/static/components/requirejs/require.js"></script>'''))
  init_notebook_mode(connected=False)
  
enable_plotly_in_cell() 
import plotly.graph_objects as go

X, Y, Z = np.mgrid[:vals.shape[0], :vals.shape[1], :vals.shape[2]]
X = X/header['d0'][0] - header['origin'][0]
Y = Y/header['d1'][1] - header['origin'][1]
Z = Z/header['d2'][2] - header['origin'][2]

fig = go.Figure(data=go.Volume(
    x=X.flatten(),
    y=Y.flatten(),
    z=Z.flatten(),
    value=vals.flatten(),
    isomin=-1,
    isomax=1,
    colorscale='BlueRed_r',
    opacity=0.1, # needs to be small to see through all surfaces
    surface_count=9, # needs to be a large number for good volume rendering
    ))
fig.show()